# Ramsey model with housing

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [27]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model description

**Write out the model in equations here.** 

Make sure you explain well the purpose of the model and comment so that other students who may not have seen it before can follow.  

In this model project, we extend the Ramsey model to include a housing market. In the first iteration, we solve the model assuming that housing supply is in fixed supply and that the housing stock does not depreciate over time. Most of the model is as described in the lecture notebook on the Ramsey model. Our changes to the mode are marked in <span style="color:red"> *red*</span>.

**Time:** Discrete and indexed by $t\in\{0,1,\dots\}$.

**Demographics:** Single infinitely-lived household.

**Household:** Households choose non-durable consumption, $C_t$ and durable housing $H_t$ such as to maximize discount lifetime utility. Labor is supplied exogenously, $N_{t}=1$, and earns a wage $w_{t}$. The return on saving is $r_{t+1}$. Utility is CRRA and given by: 

$$
\begin{aligned}
	U & =\max_{\{C_{t},\color{red}H_{t}\color{white}\}_{t=0}^{\infty}}\sum_{t=0}^{\infty}\beta^{t}\left(\frac{C_{t}^{1-\sigma}}{1-\sigma}\color{red}+\frac{H_{t}^{1-\sigma}}{1-\sigma}\color{white}\right),\beta\in(0,1),\sigma>0\\
	& \text{s.t.}\\
	A_{t}+C_t+\color{red}q_tH_t\color{black}& =(1+r_{t})A_{t-1}+w_{t}N_{t} + \color{red}q_{t}H_{t-1}\color{black}
\end{aligned}
$$

where the left hand side of the budget constraint is expenditures on saving, consumption and housing. The right hand side is what is carried over in savings from last period, labor income and the amount of housing held from the last period. 

**Firms:** Firms rent capital $K_{t-1}$ at the rental rate $r_{t}^{K}$
and hires labor $L_{t}$ at the wage rate $w_{t}$.<br>
Firms have access to the production function
    
$$
\begin{aligned}
Y_{t} &= F(K_{t-1},L_{t}) \\ &=\Gamma_t(\alpha K_{t-1}^{-\theta}+(1-\alpha)L_{t}^{-\theta})^{\frac{1}{-\theta}},\,\,\,\theta>-1,\alpha\in(0,1),\Gamma_t>0
\end{aligned}
$$

Profits are

$$
\begin{aligned}
\Pi_{t}=Y_{t}-w_{t}L_{t}-r_{t}^{K}K_{t-1}
\end{aligned}
$$

**Equilibrium:** The following conditions will be satisfied in equilibrium

1. Households maximize utility
2. Firms maximize profits
3. Labor market clear: $L_{t}=N_{t}=1$
4. Goods market clear: $Y_{t}=C_{t}+I_{t}$
5. <span style="color:red"> Housing market clear:</span>. $\color{red} \bar{H}=H_t$
6. Asset market clear: $A_{t}=K_{t}$ and $r_{t}=r_{t}^{k}-\delta$
7. Capital follows its law of motion: $K_{t}=(1-\delta)K_{t-1}+I_{t}$

**Solution**: We can summarize the model in the non-linear equation system

$$
\begin{aligned}
\boldsymbol{H}(\boldsymbol{K},\boldsymbol{C},\color{red}\boldsymbol{H}\color{white},K_{-1})=\left[\begin{array}{c}
H_{0}\\
H_{1}\\
\begin{array}{c}
\vdots\end{array}
\end{array}\right]=\left[\begin{array}{c}
0\\
0\\
\begin{array}{c}
\vdots\end{array}
\end{array}\right]
\end{aligned}
$$

where $\boldsymbol{K} = [K_0,K_1\dots]$, $\boldsymbol{C} = [C_0,C_1\dots]$,  $\color{red}\boldsymbol{H} = [H_0,H_1\dots]$ and

$$
\begin{aligned}
H_{t}
=\left[\begin{array}{c}
C_{t}^{-\sigma}-\beta(1+F_{K}(K_{t},1))C_{t+1}^{-\sigma}\\
K_{t}-[(1-\delta)K_{t-1} + F(K_{t-1},1)-C_{t}] \\
\color{red} \bar{H}^{-\sigma} + \beta C_{t+1}^{-\sigma}q_{t+1} - C_t^{-\sigma}q_t
\end{array}\right],
\end{aligned}
$$

where the last equation is derived by combining the FOC for housing demand with the assumption of fixed housing supply.

In [29]:
model = modelproject.RamseyModelClass1()
model.find_steady_state(KY_ss=(4.0,2))

initializing the model:
calling .setup()
calling .allocate()
K_ss = 4.00
Y_ss = 2.00
C_ss = 1.80
Gamma = 0.15
r_ss = 0.10
w_ss = 1.40
rk_ss = 0.15


## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 